Klasyfikacja tekstu przy pomocy metod uczenia maszynowego, na przykładzie ocen produktów spożywczych.

Źródło danych https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

In [ ]:
import numpy as np
import pandas as pd

# Ładujemy dane do dataframe, ograniczamy się do pierwszych 100 000
data = pd.read_csv(filepath_or_buffer='amazon_food_reviews.csv', sep=',', nrows=200000)
df = pd.DataFrame(data)
df.head()

In [ ]:
# Czyszczenie danych usuwam część kolumn,
df.drop(columns=["Id","ProductId", "UserId", "ProfileName", "HelpfulnessNumerator", "HelpfulnessDenominator", "Time", "Summary"], inplace=True)

# Usuwam wiersze z brakującymi danymi
df.replace("unknown", np.NAN, inplace=True)
df.dropna(inplace=True)

df.head()

Zamiana recenzji na wektory TF-IDF

In [ ]:
corpus = df['Text'].iloc[:10]
print(corpus)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vocabulary = ['product', 'good', 'has', 'food', 'have', 'of', 'bought']

pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),('tfid', TfidfTransformer())]).fit(corpus)

# Przykładowa mała macierz częstości słów
word_freq_matrix = pipe['count'].transform(corpus).toarray()

# Używamy heatmapy z biblioteki seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(word_freq_matrix, annot=True, cmap='coolwarm', xticklabels=vocabulary, yticklabels=list(range(1, len(corpus) + 1)))
plt.xlabel('Słowa')
plt.ylabel('Dokoument')
plt.title('Wystąpienia słów')
plt.show()

$$
TF(t, d) = \frac{\text{Liczba wystąpień słowa } t \text{ w dokumencie } d}{\text{Liczba wszystkich słów w dokumencie } d}
$$


Uwaga, w tym wypadku jako 'wszystkie słowa' rozumiemy które występują w słowniku

In [ ]:
word_counts_per_doc = word_freq_matrix.sum(axis=1)

# Normalizacja macierzy częstości przez liczbę słów w dokumencie
normalized_matrix = word_freq_matrix / word_counts_per_doc[:, None]

# Użyjemy seaborn heatmap do wyświetlenia znormalizowanej macierzy
plt.figure(figsize=(8, 6))
sns.heatmap(normalized_matrix, annot=True, cmap='coolwarm', xticklabels=vocabulary, yticklabels=list(range(1, len(corpus) + 1)))
plt.xlabel('Słowa')
plt.ylabel('Dokument')
plt.title('Wystąpienia słów podzielone przez liczbę słóww dokumencie')
plt.show()

$$
IDF(t, D) = \ln \left( \frac{\text{Liczba wszystkich dokumentów w zbiorze } D + 1}{\text{Liczba dokumentów, w których występuje słowo } t + 1} \right) + 1
$$

In [ ]:
idf_values = pipe['tfid'].idf_
idf_values

In [ ]:

# Konwertowanie wartości IDF do postaci macierzy (w jednym wierszu)
idf_matrix = idf_values.reshape(1, -1)

words_indices = vocabulary
# Tworzenie wykresu słupkowego dla wartości IDF
plt.figure(figsize=(10, 6))
plt.bar(words_indices, idf_values)
plt.xlabel('Słowo')
plt.ylabel('Wartość IDF')
plt.title('Inverted Document Frequency')
plt.xticks(words_indices)  # Ustawienie etykiet dla słów
plt.tight_layout()
plt.show()

$$
\text{TF-IDF}(t,D) =TF(t,d) * IDF(t, D)
$$

In [ ]:

tfidf_matrix = pipe.transform(corpus).toarray()

words_indices = vocabulary
# Tworzenie wykresu słupkowego dla wartości TF-IDF
plt.figure(figsize=(10, 6))
sns.heatmap(tfidf_matrix, cmap='viridis', annot=True, fmt='.2f', xticklabels=words_indices)
plt.title('Macierz TF-IDF')
plt.xlabel('Słowo')
plt.ylabel('Dokoument')
plt.show()

Więcej info:  
https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a  
https://www.youtube.com/watch?v=vZAXpvHhQow&t=281s

In [ ]:
# Wracając no pełnego zbioru danych
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=40000, analyzer='word', lowercase=True)

X = tfidf.fit_transform(df['Text'])
y = df['Score']

X.shape, y.shape

In [ ]:
tfidf.vocabulary_

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Pobieramy nazwy cech z TF-IDF vectorizer
feature_names = tfidf.get_feature_names_out()

# Tworzymy słownik mapujący nazwy cech do wartości TF-IDF
word_scores = dict(zip(feature_names, X[0].toarray()[0]))
# Tworzenie chmury słów
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_scores)

# Tworzenie wykresu z chmurą słów
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Następnie zeestawy dziele na treningowy zbiór i zbiór testowy

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True)
X_train.shape

Support Vector Classifiers  
https://www.youtube.com/watch?v=_YPScrckx28

In [ ]:
# Trening z LinearSVC
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Co to za liczby powyżej?  
Precision (Precyzja): Proporcja poprawnie przewidzianych pozytywnych przypadków spośród wszystkich pozytywnych przypadków przewidzianych przez model. 
$$
\text{Precision} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Positives (FP)}}
$$

Recall (Czułość): Proporcja poprawnie przewidzianych pozytywnych przypadków spośród wszystkich rzeczywiście występujących pozytywnych przypadków w danych.
$$
\text{Recall} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Negatives (FN)}}
$$

F1-Score: To średnia harmoniczna precyzji i czułości. Jest miarą równowagi między precyzją a czułością.
$$
\text{F1-score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Support (Wsparcie): Liczba wystąpień każdej z klas w zbiorze danych testowych.


In [ ]:
# Przetestujmy sobie nasz model
my_sample_food_review = "It was ok but woud not buy it again"
predicted_rating = clf.predict(tfidf.transform([my_sample_food_review]))[0]
print("Predicted rating: ", predicted_rating)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier(random_state=0)
clf_dec.fit(X_train, y_train)

In [ ]:
y_pred = clf_dec.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier
clf_neur = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2))
clf_neur.fit(X_train, y_train)

In [ ]:
y_pred = clf_neur.predict(X_test)
print(classification_report(y_test, y_pred))